I restarted the kernel after each cell!

## What I am trying to do
i.e. the following cell I want to work

In [1]:
import deep500 as d5

from deep500.frameworks import tensorflow as d5tf
from deep500.frameworks import pytorch as d5torch
from deep500.frameworks import reference as d5ref

import deep500.networks as d5nt
import deep500.datasets as d5ds



import torch
from torch.autograd import Variable
def model_to_onnx(model, batch_size, file_path, channels=1, height=64, width=64):
    dummy_input = Variable(torch.randn(batch_size, channels, height, width))

    torch.onnx.export(model, dummy_input, file_path, verbose=True)
    return file_path



# Get dataset size
mnist_shape = d5ds.dataset_shape('mnist')
classes, c, h, w = mnist_shape
BATCH_SIZE = 128

import networks
import torch.nn as nn
my_model = nn.Sequential(*networks.get_downsampling_layers(), 
                         *networks.get_residual_blocks(1), 
                         *networks.get_final_layers()) #<-- build my own model-graph


onnx_file = model_to_onnx(my_model, 128, "test.onnx")

d5_model = d5.parser.load_and_parse_model(onnx_file) #<-- trying to turn it into D500

INPUT_NODE = d5_model.get_input_nodes()[0].name
OUTPUT_NODE = d5_model.get_output_nodes()[0].name

train_set, test_set = d5ds.load_dataset('mnist', INPUT_NODE, 'labels')
d5_model.add_operation(d5.ops.LabelCrossEntropy([OUTPUT_NODE, 'labels'], 'loss'))

train_sampler = d5.ShuffleSampler(train_set, BATCH_SIZE)
test_sampler = d5.ShuffleSampler(test_set, BATCH_SIZE)

executor = d5torch.from_model(d5_model) #<<<--- here I use torch, 
    # which I have to do in order to use the adjoint method in backpropagation

optimizer = d5ref.GradientDescent(executor, 'loss', 0.1)

EPOCHS = 1
d5.test_training(executor, train_sampler, test_sampler, optimizer,
                 EPOCHS, BATCH_SIZE, OUTPUT_NODE)

graph(%0 : Float(128, 1, 64, 64)
      %1 : Float(64, 1, 3, 3)
      %2 : Float(64)
      %3 : Float(64)
      %4 : Float(64)
      %5 : Float(64)
      %6 : Float(64)
      %7 : Long()
      %8 : Float(64, 64, 4, 4)
      %9 : Float(64)
      %10 : Float(64)
      %11 : Float(64)
      %12 : Float(64)
      %13 : Float(64)
      %14 : Long()
      %15 : Float(64, 64, 2, 2)
      %16 : Float(64)
      %17 : Float(64)
      %18 : Float(64)
      %19 : Float(64)
      %20 : Float(64)
      %21 : Long()
      %22 : Float(64, 64, 3, 3)
      %23 : Float(64)
      %24 : Float(64)
      %25 : Float(64)
      %26 : Float(64)
      %27 : Float(64)
      %28 : Long()
      %29 : Float(64, 64, 3, 3)
      %30 : Float(64)
      %31 : Float(64)
      %32 : Float(64)
      %33 : Float(64)
      %34 : Float(64)
      %35 : Long()
      %36 : Float(10, 64)
      %37 : Float(10)) {
  %38 : Float(128, 64, 62, 62) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[0, 0, 0, 0], strides=[1

Testing:   0%|          | 0/78 [00:00<?, ?it/s]

RuntimeError: Only Tensors of floating point dtype can require gradients

## ERROR: Using pytorch, with the example "simple_cnn"
This gives a different error, but I think it should work

It is the code from  
from deep500/tutorials/custom_optimizer.ipynb   
but using pytorch instead of tensorflow

In [1]:
import deep500 as d5

from deep500.frameworks import tensorflow as d5tf
from deep500.frameworks import pytorch as d5torch
from deep500.frameworks import reference as d5ref

import deep500.networks as d5nt
import deep500.datasets as d5ds



# Get dataset size
mnist_shape = d5ds.dataset_shape('mnist')
classes, c, h, w = mnist_shape
BATCH_SIZE = 128

onnx_file = d5nt.export_network("simple_cnn", BATCH_SIZE, classes=classes,
                             channels=c, height=h, width=w)

d5_model = d5.parser.load_and_parse_model(onnx_file)

INPUT_NODE = d5_model.get_input_nodes()[0].name
OUTPUT_NODE = d5_model.get_output_nodes()[0].name

train_set, test_set = d5ds.load_dataset('mnist', INPUT_NODE, 'labels')
d5_model.add_operation(d5.ops.LabelCrossEntropy([OUTPUT_NODE, 'labels'], 'loss'))

train_sampler = d5.ShuffleSampler(train_set, BATCH_SIZE)
test_sampler = d5.ShuffleSampler(test_set, BATCH_SIZE)

executor = d5torch.from_model(d5_model) #<<<--- here I use pytorch

optimizer = d5ref.GradientDescent(executor, 'loss', 0.1)

EPOCHS = 1
d5.test_training(executor, train_sampler, test_sampler, optimizer,
                 EPOCHS, BATCH_SIZE, OUTPUT_NODE)

/Users/clemens/.pyenv/versions/3.7.0/envs/neural_ode/lib/python3.7/site-packages/deep500/networks/pytorch_mnist.py:11: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(m.weight, gain=np.sqrt(2))
/Users/clemens/.pyenv/versions/3.7.0/envs/neural_ode/lib/python3.7/site-packages/deep500/networks/pytorch_mnist.py:12: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias, 0)


graph(%0 : Float(128, 1, 28, 28)
      %1 : Float(10, 1, 5, 5)
      %2 : Float(10)
      %3 : Float(20, 10, 5, 5)
      %4 : Float(20)
      %5 : Float(50, 320)
      %6 : Float(50)
      %7 : Float(10, 50)
      %8 : Float(10)) {
  %9 : Float(128, 10, 24, 24) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[5, 5], pads=[0, 0, 0, 0], strides=[1, 1]](%0, %1, %2), scope: Net/Conv2d[conv1]
  %10 : Float(128, 10, 12, 12) = onnx::MaxPool[kernel_shape=[2, 2], pads=[0, 0, 0, 0], strides=[2, 2]](%9), scope: Net
  %11 : Float(128, 10, 12, 12) = onnx::Relu(%10), scope: Net
  %12 : Float(128, 20, 8, 8) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[5, 5], pads=[0, 0, 0, 0], strides=[1, 1]](%11, %3, %4), scope: Net/Conv2d[conv2]
  %13 : Float(128, 20, 4, 4) = onnx::MaxPool[kernel_shape=[2, 2], pads=[0, 0, 0, 0], strides=[2, 2]](%12), scope: Net
  %14 : Float(128, 20, 4, 4) = onnx::Relu(%13), scope: Net
  %15 : Tensor = onnx::Constant[value=  -1  320 [ CPULongType{2} ]](), scope: Net
 

Training (epoch 1/1):   0%|          | 0/468 [00:00<?, ?it/s]

Trying to fetch a None tensor grad_1


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

### The same code but using tensorflow || this works
from deep500/tutorials/custom_optimizer.ipynb

and using the framework tensorflow

In [1]:
import deep500 as d5

from deep500.frameworks import tensorflow as d5tf
from deep500.frameworks import pytorch as d5torch
from deep500.frameworks import reference as d5ref

import deep500.networks as d5nt
import deep500.datasets as d5ds


# Get dataset size
mnist_shape = d5ds.dataset_shape('mnist')
classes, c, h, w = mnist_shape
BATCH_SIZE = 128

# model = nn.Sequential(*get_downsamplint_layers(), *get_final_layers())

# onnx_file = model_to_onnx(model, 128, "test.onnx")

onnx_file = d5nt.export_network("simple_cnn", BATCH_SIZE, classes=classes,
                             channels=c, height=h, width=w)

d5_model = d5.parser.load_and_parse_model(onnx_file)

INPUT_NODE = d5_model.get_input_nodes()[0].name
OUTPUT_NODE = d5_model.get_output_nodes()[0].name

train_set, test_set = d5ds.load_dataset('mnist', INPUT_NODE, 'labels')
d5_model.add_operation(d5.ops.LabelCrossEntropy([OUTPUT_NODE, 'labels'], 'loss'))

train_sampler = d5.ShuffleSampler(train_set, BATCH_SIZE)
test_sampler = d5.ShuffleSampler(test_set, BATCH_SIZE)

executor = d5tf.from_model(d5_model) #<<<--- here I use tensorflow

optimizer = d5ref.GradientDescent(executor, 'loss', 0.1)

EPOCHS = 1
d5.test_training(executor, train_sampler, test_sampler, optimizer,
                 EPOCHS, BATCH_SIZE, OUTPUT_NODE)

/Users/clemens/.pyenv/versions/3.7.0/envs/neural_ode/lib/python3.7/site-packages/deep500/networks/pytorch_mnist.py:11: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(m.weight, gain=np.sqrt(2))
/Users/clemens/.pyenv/versions/3.7.0/envs/neural_ode/lib/python3.7/site-packages/deep500/networks/pytorch_mnist.py:12: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias, 0)


graph(%0 : Float(128, 1, 28, 28)
      %1 : Float(10, 1, 5, 5)
      %2 : Float(10)
      %3 : Float(20, 10, 5, 5)
      %4 : Float(20)
      %5 : Float(50, 320)
      %6 : Float(50)
      %7 : Float(10, 50)
      %8 : Float(10)) {
  %9 : Float(128, 10, 24, 24) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[5, 5], pads=[0, 0, 0, 0], strides=[1, 1]](%0, %1, %2), scope: Net/Conv2d[conv1]
  %10 : Float(128, 10, 12, 12) = onnx::MaxPool[kernel_shape=[2, 2], pads=[0, 0, 0, 0], strides=[2, 2]](%9), scope: Net
  %11 : Float(128, 10, 12, 12) = onnx::Relu(%10), scope: Net
  %12 : Float(128, 20, 8, 8) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[5, 5], pads=[0, 0, 0, 0], strides=[1, 1]](%11, %3, %4), scope: Net/Conv2d[conv2]
  %13 : Float(128, 20, 4, 4) = onnx::MaxPool[kernel_shape=[2, 2], pads=[0, 0, 0, 0], strides=[2, 2]](%12), scope: Net
  %14 : Float(128, 20, 4, 4) = onnx::Relu(%13), scope: Net
  %15 : Tensor = onnx::Constant[value=  -1  320 [ CPULongType{2} ]](), scope: Net
 

Training (epoch 1/1):   0%|          | 0/468 [00:00<?, ?it/s]

Instructions for updating:
Use tf.cast instead.


Testing: 100%|██████████| 78/78 [00:00<00:00, 99.93it/s, accuracy=96.5, test_loss=0.107]

TrainingAccuracy: 85.81396901709401
TestAccuracy: 96.47435897435898


[85.81396901709401, 96.47435897435898]